In [1]:
from pyspark import SparkContext
import json
from time import time

In [2]:
path = "/Users/liangsiqi/Documents/Dataset/yelp_dataset/"
user_file = "user.json"

In [3]:
def iterm_num(iterator):
    cnt = 0
    for x in iterator:
        cnt += 1
    yield cnt

In [4]:
sc = SparkContext()
start1 = time()
userRDD_default = sc.textFile(path + user_file)
default_n_partition = userRDD_default.getNumPartitions()
default_n_items = userRDD_default.mapPartitions(iterm_num).collect()
userRDD_json_default = userRDD_default.map(json.loads)
review_cnt_RDD = userRDD_json_default.map(lambda x: [x['user_id'], x['review_count']])
most_review = review_cnt_RDD.sortBy(lambda x: x[1], ascending=False).take(10)
end1 = time()
default_exe_time = int(end1 - start1)
sc.stop()
print("Time for default %d partitions: %d s" % (default_n_partition, default_exe_time))

Time for default 74 partitions: 21 s


In [9]:
def json_parser(iterator):
    yield [json.loads(x) for x in iterator]

def review_key_value(iterator):
    yield [(x['user_id'], x['review_count']) for x in iterator]

In [37]:
def sorted_within_partition(iterator):
    yield sorted(iterator,key=lambda x: -x[1],reverse=False)

def top10_value(iterator):
    iterms_list = list(iterator)[0]
    if len(iterms_list)<10:
        yield iterms_list
    else:
        yield iterms_list[:10]

In [47]:
sc.stop()
sc = SparkContext()
customized_n_partition = 74
start2 = time()
userRDD_customized = sc.textFile(path + user_file, customized_n_partition)
print("Customized partition number: %d" % userRDD_customized.getNumPartitions())
customized_n_items = userRDD_customized.mapPartitions(iterm_num)

review_cnt_RDD_customized = userRDD_customized.map(json.loads).map(lambda x:[x['user_id'], x['review_count']])
print(review_cnt_RDD_customized.getNumPartitions())
sorted_withn_partitionRDD = review_cnt_RDD_customized.mapPartitions(sorted_within_partition)
top10_withn_partitionRDD = sorted_withn_partitionRDD.mapPartitions(top10_value)
top10_reviews = top10_withn_partitionRDD.flatMap(lambda x: x).takeOrdered(10,key=lambda x: -x[1])
end2 = time()
customized_exe_time = int(end2 - start2)
print("Time for customized %d partitions: %d s" % (customized_n_partition, customized_exe_time))

Customized partition number: 74
74
Time for customized 74 partitions: 4 s


In [45]:
top10_withn_partitionRDD.takeOrdered(10,key=lambda x: -x[1])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 7.0 failed 1 times, most recent failure: Lost task 3.0 in stage 7.0 (TID 249, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "//anaconda3/envs/inf553/lib/python3.6/site-packages/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "//anaconda3/envs/inf553/lib/python3.6/site-packages/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "//anaconda3/envs/inf553/lib/python3.6/site-packages/pyspark/rd

In [8]:
end2 = time()
sc.stop()


1637138

In [48]:
explanation = ['Without customized partition functions, the program ',
                   'tries to sort all 1637138 users first then get the ',
                   'top-10 users; while using customized partition ',
                   'functions, the program first sorted within each ',
                   'partition and get top-10 for each partition, then ',
                   'sort  10 x n_partition  users  and output the final top-10 ',
                   'result. The default method takes much longer time ',
                   'as well as takes more memory for storing intermediate ',
                   '(Key,Value) results.']
''.join(explanation)

'Without customized partition functions, the program tries to sort all 1637138 users first then get the top-10 users; while using customized partition functions, the program first sorted within each partition and get top-10 for each partition, then sort  10 x n_partition  users  and output the final top-10 result. The default method takes much longer time as well as takes more memory for storing intermediate (Key,Value) results.'